In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma, FAISS
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()

True

In [20]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-Guard-3-1B-INT4", 
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)


llm = ChatHuggingFace(llm=llm)



# messages = [
#     SystemMessage(content="You are a helpful assistant that explains complex topics simply."),
#     HumanMessage(content="How does vision transformers work?"),
# ]

# response = llm.invoke(messages)
# print(response.content)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [15]:
pdf_path = r'E:\pandas\MLOPS\RAG Mlflow\data\3 laws.pdf'

In [16]:
def load_process_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    for page in pages:
        page.page_content = page.page_content.replace('\n', ' ')
        page.metadata = {"source": pdf_path,
            "subject": "49 law of power",
            "doc_id": "syllabus_001" }

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_documents(pages)
    return chunks


In [17]:
chunks = load_process_pdf(pdf_path)

In [18]:
print(f"Total chunks created: {len(chunks)}")
print(f"\nSample chunk metadata:")
print(chunks[0].metadata)
print(f"\nSample chunk content (first 200 chars):")
print(chunks[0].page_content[:200])

Total chunks created: 73

Sample chunk metadata:
{'source': 'E:\\pandas\\MLOPS\\RAG Mlflow\\data\\3 laws.pdf', 'subject': '49 law of power', 'doc_id': 'syllabus_001'}

Sample chunk content (first 200 chars):
LAW 1 NEVER OUTSHINE THE MASTER JUDGMENT Always make those above you feel comfortably superior. In your desire to please and impress them, do not go too far in displaying your talents or you might acc


In [3]:
prompt = PromptTemplate(
        input_variables=["context", "input"],
        template=(
            '''You are an exam assistant. Answer ONLY from the given context.If the answer is not in the context, say:
                "Not found in the syllabus.".\n\n'''
            "Context:\n{context}\n\n"
            "Question: {input}"
        )
    )


In [27]:
PERSIST_DIRECTORY = "./faiss_db"

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

vectorstore = FAISS.from_documents(chunks, embeddings, persist_directory=PERSIST_DIRECTORY)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

NameError: name 'retriever' is not defined